In [5]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [6]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

In [7]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults(max_results=3) # create a search AI agents
search.invoke("Who are the top stars of the 2024 Eurocup?")

[{'url': 'https://www.nbcsports.com/soccer/news/euro-2024-player-power-rankings-who-are-the-top-20-stars',
  'content': 'With Kylian Mbappe, Harry Kane and Kevin de Bruyne aiming to lead their teams to glory, plus ageing superstars like Toni Kroos and Cristiano Ronaldo still going strong and young stars Jude Bellingham, Florian Wirtz and Lamine Yamal set to take over, you can look all over the place for star performances this summer.\n\nPlus, every tournament there are always stars who emerge to make a name for themselves and players from Georgia and Austria are standing tall. [...] 10. Cody Gakpo (Netherlands)  \n9. Kevin de Bruyne (Belgium)  \n8. William Saliba (France)  \n7. Bruno Fernandes (Portugal)  \n6. Kylian Mbappe (France)\n\n5. Marcel Sabitzer (Austria)  \n4. Gianluigi Donnarumma (Italy)  \n3. Jamal Musiala (Germany)  \n2. Florian Wirtz (Germany)  \n1. Fabian Ruiz (Spain) [...] Below are the top 20 players at EURO 2024 based on current form and we will update it throughout t

In [8]:
tools = [search]

In [9]:
tools

[TavilySearchResults(max_results=3)]

In [ ]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools) # the agent decide whether it's needed to use the search or not

In [11]:
from langchain_core.messages import HumanMessage

response = agent_executor.invoke({"messages": [HumanMessage(content="Where is the soccer Eurocup 2024 played?")]})

response["messages"]

[HumanMessage(content='Where is the soccer Eurocup 2024 played?', id='0e30ab67-4b8b-4503-8021-4c865a0ee42f'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_2IJKJMsjl5SnSdBs9Avjay6W', 'function': {'arguments': '{"query":"Eurocup 2024 location"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 90, 'total_tokens': 113, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_376a7ccef1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-58c96305-c677-4644-8243-38fc5d119143-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Eurocup 2024 location'}, 'id': 'call_2IJKJMsjl5SnSdBs9Avjay6W', 'type': 'tool_call'}], usage_metadata={'input_t

In [12]:
from langchain_core.messages import HumanMessage

# This one's not gonna use the search
response = agent_executor.invoke({"messages": [HumanMessage(content="What is Python?")]}) 

response["messages"]

[HumanMessage(content='What is Python?', id='f22b97d4-8004-4a35-845d-08d25af7cad7'),
 AIMessage(content='Python is a high-level, interpreted programming language known for its simplicity and readability. It was created by Guido van Rossum and first released in 1991. Python supports multiple programming paradigms, including procedural, object-oriented, and functional programming. It is widely used for web development, data analysis, artificial intelligence, scientific computing, automation, and more. Python has a large standard library and an active community, which contribute to its extensive ecosystem of third-party packages and frameworks.', response_metadata={'token_usage': {'completion_tokens': 96, 'prompt_tokens': 83, 'total_tokens': 179, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14',

In [13]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="When and where will it be the 2024 Eurocup final match?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_LnoSDRZxinRp7iW2J3b3Pgq4', 'function': {'arguments': '{"query":"2024 Eurocup final match date and location"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 94, 'total_tokens': 120, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_376a7ccef1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-1eca1647-941b-4eef-ac4a-bb1f62924f83-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2024 Eurocup final match date and location'}, 'id': 'call_LnoSDRZxinRp7iW2J3b3Pgq4', 'type': 'tool_call'}], usage_metadata={'input_tokens': 94, 'output_tokens': 26, 'total_token

# Adding memory

In [14]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [19]:
agent_executor = create_react_agent(llm, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "001"}}

In [20]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Who won the 2024 soccer Eurocup?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Spain won the 2024 soccer Eurocup, defeating England 2-1 in the final to claim their record fourth European Championship title.', response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 944, 'total_tokens': 973, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_376a7ccef1', 'finish_reason': 'stop', 'logprobs': None}, id='run-9cd8c416-321c-4eb8-b5ba-115f7aa47f12-0', usage_metadata={'input_tokens': 944, 'output_tokens': 29, 'total_tokens': 973})]}}
----


In [21]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Who were the top stars of that winner team?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="The top stars of the Spain team that won the 2024 Eurocup included Nico Williams, who scored an important goal early in the second half, and Lamine Yamal, a freshly turned 17-year-old who provided a key assist. Additionally, Mikel Oyarzabal was a crucial player who scored the winning goal in the 87th minute. These players were standout performers in Spain's successful campaign.", response_metadata={'token_usage': {'completion_tokens': 85, 'prompt_tokens': 990, 'total_tokens': 1075, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_376a7ccef1', 'finish_reason': 'stop', 'logprobs': None}, id='run-0e83de4a-0b2b-4094-bb0d-35bc0f1b14d4-0', usage_metadata={'input_tokens': 990, 'output_tokens': 85, 'total_tokens': 1075})]}}
----


In [ ]:
config = {"configurable": {"thread_id": "002"}} # user 2, so no memory of user 1's chat history
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="About what soccer team we were talking?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="We have not mentioned any specific soccer team in our conversation so far. If you tell me the name of the team you're interested in, I can provide information about their top stars.", response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 167, 'total_tokens': 204, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_376a7ccef1', 'finish_reason': 'stop', 'logprobs': None}, id='run-157efecc-b071-456c-8f3f-c72602da2152-0', usage_metadata={'input_tokens': 167, 'output_tokens': 37, 'total_tokens': 204})]}}
----
